В этом задании вам предстоит собрать данные с сайта lifehacker.ru из рубрики Технологии с помощью библиотек requests и BeautifulSoup.
В частности, нужно собрать заголовки и тексты материалов с первых десяти страниц рубрики.

Примерный пайплайн решения задания аналогичен примеру из 🔗 itmo_course_python_07_scraping_bs4.ipynb:
Определить формат ссылки для пагинации - 1 балл

В разметке страницы найти уникальные классы/идентификаторы блока с названием и содержанием материала- 3 балла

Получить содержимое десяти страниц списка материалов, выделить из него ссылки на каждый материал - 2 балла

Перебрать все полученные ссылки и получить html-код каждого материала - 1 балл

Распарсить полученный текст разметки с помощью BeautifulSoup, вытащив по каждой ссылке заголовок и содержание материала - 2 балл

Создать датайфрейм с полученными данными - 1 балл

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

### Получение всего контента страницы

In [11]:
response = requests.get('https://lifehacker.ru/topics/technology/')
response.text

'<!DOCTYPE html><html  lang="ru-RU" data-capo=""><head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<title>Технологии — Лайфхакер</title>\n<link rel="preconnect" href="https://www.googletagmanager.com">\n<link rel="preconnect" href="https://top-fwz1.mail.ru">\n<link rel="preconnect" href="https://mc.yandex.ru">\n<link rel="preconnect" href="https://static.criteo.net">\n<link rel="preconnect" href="https://exchange.buzzoola.com">\n<link rel="preconnect" href="https://pb.adriver.ru">\n<link rel="preconnect" href="https://ad.mail.ru">\n<script>window.yaContextCb = window.yaContextCb || [];\nwindow.Ya || (window.Ya = {});\nwindow.Ya.adfoxCode || (window.Ya.adfoxCode = {});\nwindow.Ya.adfoxCode.hbCallbacks || (window.Ya.adfoxCode.hbCallbacks = []);</script>\n<script src="https://yandex.ru/ads/system/context.js" crossorigin="anonymous" async></script>\n<script src="https://yandex.ru/ads/system/header-bidding.js" async></script>\n<style>@font-f

In [12]:
base_url = 'https://lifehacker.ru/topics/technology/' # базовая часть ссылки, позже в цикле будем добавлять к ней пагинацию
response = requests.get('https://lifehacker.ru/topics/technology/?page=1') # получаем контент первой страиниц
soup = BeautifulSoup(response.text, 'lxml') # инициализируем объект bs4 и задаем парсер lxml

In [13]:
raw_items = soup.find_all('a', class_='lh-small-article-card__link')  # ищем все a-элементы с классом tm-article-comments-counter-link__link
links = ['https://lifehacker.ru' + item.get('href') for item in raw_items if item.get('href')] # получаем у ссылок только href-атрибут, убираем подстроку '/comments'
links

['https://lifehacker.ru/neochevidnye-funkcii-umnyx-kolonok/',
 'https://lifehacker.ru/vzlom-chat-bota/',
 'https://lifehacker.ru/vyxod-hyperos-2-2/',
 'https://lifehacker.ru/xiaomi-maska-dlya-sna-s-massazhem/',
 'https://lifehacker.ru/chrome-smozhet-sam-menyat-paroli/',
 'https://lifehacker.ru/anons-project-aura/',
 'https://lifehacker.ru/zapusk-ii-poiska-google/',
 'https://lifehacker.ru/anons-google-flow/',
 'https://lifehacker.ru/kak-vypryamit-skruchennyi-kabel/',
 'https://lifehacker.ru/sinxronnyi-perevod-v-meet/',
 'https://lifehacker.ru/s-chego-nachalsya-internet/',
 'https://lifehacker.ru/kak-eto-rabotaet-umnoe-shumopodavlenie-v-naushnikax/',
 'https://lifehacker.ru/anons-msi-claw-a8/',
 'https://lifehacker.ru/ii-deistviya-v-provodnike/',
 'https://lifehacker.ru/vyxod-one-ui-7/',
 'https://lifehacker.ru/iphone-17-air-i-iphone-16-plus/',
 'https://lifehacker.ru/anons-huawei-nova-14/',
 'https://lifehacker.ru/spat-s-telefonom-na-zaryadke/',
 'https://lifehacker.ru/anons-huawei-mat

Теперь мы получили ссылки на 20 статей с первой страницы Хабра, осталось собрать ссылки со всех 10 страниц

In [14]:
import requests
requests.get('https://lifehacker.ru/xiaomi-maska-dlya-sna-s-massazhem/').text

'<!DOCTYPE html><html  lang="ru-RU" data-capo=""><head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<title>Xiaomi представила недорогую маску для сна — с массажем и подогревом — Лайфхакер</title>\n<link rel="preconnect" href="https://www.googletagmanager.com">\n<link rel="preconnect" href="https://top-fwz1.mail.ru">\n<link rel="preconnect" href="https://mc.yandex.ru">\n<link rel="preconnect" href="https://static.criteo.net">\n<link rel="preconnect" href="https://exchange.buzzoola.com">\n<link rel="preconnect" href="https://pb.adriver.ru">\n<link rel="preconnect" href="https://ad.mail.ru">\n<script>window.yaContextCb = window.yaContextCb || [];\nwindow.Ya || (window.Ya = {});\nwindow.Ya.adfoxCode || (window.Ya.adfoxCode = {});\nwindow.Ya.adfoxCode.hbCallbacks || (window.Ya.adfoxCode.hbCallbacks = []);</script>\n<script src="https://yandex.ru/ads/system/context.js" crossorigin="anonymous" async></script>\n<script src="https://yandex.ru/ads

In [15]:
base_url = 'https://lifehacker.ru/topics/technology/'
parsed_urls = []

for page_num in range(1, 11):
    url = f'{base_url}?page={page_num}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    raw_items = soup.find_all('a', class_='lh-small-article-card__link')
    parsed_urls += ['https://lifehacker.ru' + item.get('href') for item in raw_items if item.get('href')]

len(parsed_urls)

300

In [16]:
response = requests.get('https://lifehacker.ru/gpt-4-1-v-chatgpt/')
soup = BeautifulSoup(response.text, 'lxml')
soup.find('article', class_='single-article__post-content single-article__content-container').text

'OpenAI добавила свои новые языковые модели GPT‑4.1 и GPT‑4.1 mini в ChatGPT. Обновление ориентировано в первую очередь на разработчиков: модели заметно лучше справляются с написанием и отладкой кода.GPT‑4.1 уже доступна подписчикам тарифов ChatGPT Plus, Pro и Team. Модель отличается высокой скоростью работы и, по данным OpenAI, превосходит GPT-4o в задачах, связанных с программированием. Облегчённая версия — GPT‑4.1 mini — предназначена как для бесплатных, так и для платных пользователей. С выходом новинок компания полностью прекратила поддержку GPT‑4.0 mini и удалила её из интерфейса.Обе модели представили ещё в апреле 2025 года, но сначала они были доступны лишь через API. Тогда многие пользователи остались недовольны политикой OpenAI, поскольку та вместе с релизом не выпустила отчёт о безопасности, что восприняли как снижение стандартов прозрачности. Руководитель отдела безопасности OpenAI Йоханнес Хайдеке ответил, что GPT‑4.1 «не добавляет новых модальностей и не превосходит GPT-4

In [23]:
result = []

for url in tqdm(parsed_urls):
    article = {}
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')

        # Заголовок
        title_tag = soup.find('h1', class_='article-card__title')
        article['title'] = title_tag.text.strip() if title_tag else 'Заголовок не найден'

        # Текст
        text_tag = soup.find('article', class_='single-article__post-content single-article__content-container')
        article['text'] = text_tag.text.strip() if text_tag else 'Текст не найден'

    except Exception as e:
        article['title'] = 'Ошибка'
        article['text'] = f'Ошибка: {e}'
        article['votes_count'] = '0'

    result.append(article)

100%|██████████| 300/300 [11:04<00:00,  2.21s/it]


Сделаем красивый датафрейм

In [24]:
pd.set_option('display.max_colwidth', 400)

In [25]:
data = pd.DataFrame(result)
data.head()

,title,text
0,"8 функций умных колонок, о которых вы могли не знать","1. Детский режимКолонки позволяют фильтровать контент «для взрослых» и ограничивать поиск в интернете, когда к голосовому ассистенту обращаются дети. Режим настраивается по-разному в зависимости от производителя колонки:«Яндекс Станции»: в приложении «Дом с Алисой» откройте настройки и выберите «Алиса для детей» → «Ограничение контента». Можно выставить, будет ли ассистент отличать детские гол..."
1,Исследование: большинство чат-ботов легко обмануть и заставить давать опасные ответы,"Исследователи из Университета Бен-Гуриона в Израиле предупредили, что взломанные чат-боты могут делиться незаконной информацией, усвоенной во время обучения. Это заявление прозвучало на фоне растущей тенденции обхода пользователями встроенных механизмов безопасности.Большие языковые модели, такие как ChatGPT, Gemini и Claude, функционируют на основе массивов данных, собранных из интернета. Нес..."
2,"Стабильное обновление Xiaomi HyperOS 2.2 готово для 12 устройств, включая Xiaomi 14 и Redmi K70","Xiaomi готовится к выпуску стабильной версии HyperOS 2.2. По данным XiaomiTime, официальный релиз запланирован на 22 мая. Развёртывание обновления начнётся с 11 моделей смартфонов и одного планшета:Poco F7 / Redmi Turbo 4 Pro;\nPoco M7 Pro 5G / Redmi Note 14;\nPoco X7 / Redmi Note 14 Pro;\nRedmi K70 / Poco F6 Pro;\nRedmi K70E / Poco X6 Pro;\nXiaomi 14;\nXiaomi 14 Pro;\nXiaomi 14 Ultra;\nXiaomi..."
3,Xiaomi представила недорогую маску для сна — с массажем и подогревом,"Xiaomi выпустила новый гаджет для расслабления и сна — массажную маску для глаз Mijia. Она не только защищает от света, но и оснащена функциями массажа и подогрева.Изображение: Xiaomi В конструкции используются четыре вибромоторчика: два массируют зону под глазами, два — над бровями. Управлять интенсивностью можно вручную кнопками или с помощью приложения Mijia Home. Также предусмотрено шесть ..."
4,Chrome сможет сам менять ненадёжные пароли пользователей,"Google добавит в Chrome функцию, которая сможет сама сгенерировать и поменять пароль на сайте, если браузер посчитает его слабым или скомпрометированным — в один клик. Происходить это будет только с согласия пользователя.Изображение: GoogleНововведение встроят в существующий менеджер паролей браузера, который и раньше предупреждал о небезопасных комбинациях. Однако, как признала Париса Табриз,..."
